In [1]:
import pandas as pd 


In [ ]:
for i in range(1, 5):
    print("abc" + str(i) + ".csv")

In [2]:
admissions = pd.read_csv('/Volumes/Oussama/project/Data_comp/ADMISSIONS.csv')
d_reduced_chartitems = pd.read_csv('/Volumes/Oussama/project/Data_comp/D_CHARTITEMS.csv')


In [3]:
complete_chart_events = pd.DataFrame()
d_reduced_chartitems


,MEASUREMENT,ITEMID
0,Heart Rate 1,211
1,Heart Rate 2,220045
2,Heart rate Alarm - High,220046
3,Heart Rate Alarm - Low,220047
4,Mean Arterial Pressure 1,224322
...,...,...
57,Height cm,226730
58,Creatinine 1,1525
59,Creatinine 2,220615
60,Arterial Blood Pressure mean,220052


In [4]:
for i in range(1, 332):
    chart_events = pd.read_csv('/Volumes/Oussama/project data/chartevents/CHARTEVENTS_' + str(i) + '.csv')
    # remove redundant or irrelevant data
    chart_events.drop(['ROW_ID', 'STORETIME', 'CGID', 'VALUEUOM', 'VALUE', 'RESULTSTATUS', 'STOPPED'], axis=1, inplace=True)
    chart_events['CHARTTIME'] = pd.to_datetime(chart_events['CHARTTIME'])
    # remove data that is not part of the investigation
    chart_events = chart_events[chart_events['SUBJECT_ID'].isin(admissions['SUBJECT_ID'])] 
    chart_events = chart_events[chart_events['ITEMID'].isin(d_reduced_chartitems['ITEMID'])]
    chart_events = chart_events[chart_events['ERROR'] != 1.0]
    chart_events = chart_events.drop(['ERROR'], axis=1)
    # drop NANs
    chart_events.dropna(subset=['VALUENUM'], inplace=True)
    chart_events['FLAG'] = chart_events['WARNING'].apply(lambda x: 1 if (x == 1.0) else 0)
    chart_events['FLAG'] = chart_events['FLAG'].astype('category')
    chart_events.drop(['WARNING'], axis=1, inplace=True)
    complete_chart_events = pd.concat([complete_chart_events, chart_events], ignore_index=True)
    

    

/Users/oussamagharsellaoui/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/oussamagharsellaoui/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,10,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/oussamagharsellaoui/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
complete_chart_events.dropna(subset=['VALUENUM'], inplace=True)  


In [6]:
complete_chart_events.drop_duplicates(inplace=True)


In [7]:
complete_chart_events.fillna({'FLAG':0}, inplace=True)
complete_chart_events['FLAG'] = complete_chart_events['FLAG'].astype('category')
complete_chart_events.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28385140 entries, 0 to 28385237
Data columns (total 7 columns):
 #   Column      Dtype         
---  ------      -----         
 0   SUBJECT_ID  int64         
 1   HADM_ID     int64         
 2   ICUSTAY_ID  float64       
 3   ITEMID      int64         
 4   CHARTTIME   datetime64[ns]
 5   VALUENUM    float64       
 6   FLAG        category      
dtypes: category(1), datetime64[ns](1), float64(2), int64(3)
memory usage: 1.5 GB


In [8]:
complete_chart_events.to_csv('/Volumes/Oussama/project/Data_comp/CHARTEVENTS.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
complete_chart_events